In [2]:
import sys
sys.path.append('../build')

import os
if os.name == 'nt':
    os.add_dll_directory('C:\\Windows\\SysWOW64')

Import the modules to be used:

In [3]:
import information_reconciliation as ir
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl

Define the variables to required for reconciliation.

In [ ]:
N = 1.6e7
max_decoding_iteration = 500
d = 8 # Reconciliation dimension
code_rate = 0.2
SNR_min = -4.8
SNR_max = -4.45
total_error_count = 256 # Total number of frame errors

In [ ]:
statistics_array = []
display=ir.print() # Display object for printing statistics

for SNR in np.arange(SNR_min, SNR_max, 0.05):
    statistics_total = ir.statistics()
    sigma_square = 10**(-SNR/10) #noise variance


    while statistics_total.get_total_frame_error_count()<total_error_count:
        X = np.random.normal(0, 1, int(N))
        Y = X + np.sqrt(sigma_square)*np.random.normal(0, 1, int(N))

        stats = ir.reconcile(alice_states=X, bob_states= Y, rate= code_rate, noise_variance = sigma_square, NoI=max_decoding_iteration, MDR_dim= d, fast_decoding= 1, layered_decoding= 1)
        
        statistics_total = statistics_total + stats
    
    statistics_array.append(statistics_total)

Save the simulation results

In [4]:
def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pkl.dump(obj, outp, pkl.HIGHEST_PROTOCOL)

In [ ]:
save_object(statistics_array, 'statistics_array.pkl')
save_object(np.arange(SNR_min, SNR_max, 0.05), 'SNR_array.pkl')

Plot the simulation results

In [ ]:
plt.figure()
plt.plot(np.arange(SNR_min, SNR_max, 0.05),[statistics_array[i].get_frame_error_rate() for i in range(len(statistics_array))], label='FER')
plt.plot(np.arange(SNR_min, SNR_max, 0.05),[statistics_array[i].get_bit_error_rate() for i in range(len(statistics_array))], linestyle='dashed', label='BER')
plt.yscale('log')
plt.xlabel('SNR [dB]')
plt.ylabel('FER / BER')
plt.legend()
plt.grid()
plt.show()